In [1]:
import requests
import os
import time
from bs4 import BeautifulSoup
from urllib.request import Request, urlopen
import pymongo
from fastapi import FastAPI
from datetime import datetime
from pymongo import MongoClient

In [17]:
class Best_buy_scrap:
    def __init__(self , product_name):
        self.search_term = product_name
        
    ### request search page information and return bs4 object
    def get_search_pages(self , search_term , p_number):
        ### URL
        search_page_url = "https://www.bestbuy.com/site/searchpage.jsp?st="
        ### search product
        search_product = search_term
        
        ### page_number
        page_number = 0
        if p_number == 0:
            page_number = 1
        else:
            page_number = p_number

        ### page_number search term
        search_page_encoding = f"&cp={page_number}"

        ### User agent
        User_Agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36 Edg/88.0.705.56'

        ### request access
        r = requests.get(search_page_url + search_product + search_page_encoding , headers = {'User-Agent' : User_Agent})
        
        ### print url's
        print(search_page_url + search_product + search_page_encoding)

        ### Assign bs4 object to soup
        soup = BeautifulSoup(r.content, 'lxml')
        
        #creating a file "ebay_amazon gift cards_webpages" for storing all the pages of seacrh results that we download. But before that we are checking if the we already have one file with the same name. If so they code piece will not run
        try:
            if not os.path.exists("best_buy_folder"):
                os.makedirs("best_buy_folder")
        except:
            pass
        
        #downloading the first pages of seacrh results 

        response_page1 = requests.get(search_page_url + search_product + search_page_encoding , headers = {'User-Agent' : User_Agent})
        soup_page1 = BeautifulSoup(response_page1.content, 'html.parser')
        with open(f'best_buy_folder/best_buy_page_{page_number}.html', 'w', encoding='utf-8') as file:
            file.write(str(soup_page1))

        return soup
    
    ### handle price text
    def handle_price(self , price):
        try:
            price = price.replace("$" , "")
            price = float(price)
        except:
            pass
        return price

    ### exrtract brand name from header
    def get_brand(self , header):
        try:
            brand = header.split("-")[0].strip(" ")
        except:
            brand = ""
        return brand
    
    ### get product info from bs4 object and return dict of info
    def get_product_info(self , product):
        return_dict = {}

        ### get header
        header = product.find('h4').text
        return_dict.update({'header' : header})

        ### get brand
        brand = self.get_brand(header)
        return_dict.update({'brand' : brand})

        ### get labels
        try:
            labels = product.find('div' , attrs = {'class' : 'lv-stacked-carousel'}).find_all('button')
            labels = [lab.text for lab in labels]
        except:
            labels = []
        return_dict.update({'labels' : labels})


        ### get model
        model = product.find_all('span' , attrs = {'class' : 'sku-value'})[0].text
        return_dict.update({'model' : model})

        ### get sku
        sku = product.find_all('span' , attrs = {'class' : 'sku-value'})[1].text
        return_dict.update({'sku' : sku})

        ### get stars
        try:
            stars = product.find('p').text.split(' ')[1]
            return_dict.update({'stars' : float(stars)})
        except:
            pass 

        ### get number of reviews
        number_reviews = product.find('p').text.split(' ')[-2]
        return_dict.update({'number_of_reivews' : number_reviews})

        ### get price
        try:
            price = product.find("div" , attrs = {"class" : "priceView-hero-price priceView-customer-price"}).find('span').text
            return_dict.update({"price" : self.handle_price(price)})
        except:
            pass

        ### get old price
        try:
            old_price = product.find("div" , attrs = {"class" : "pricing-price__regular-price"}).text.split(' ')[1]
            return_dict.update({'original_price' : handle_price(old_price)})
        except:
            pass

        ### price diff
        try:
            reduction = handle_price(old_price) - handle_price(price)
            return_dict.update({'price_reduction' : reduction})
        except:
            pass

        return return_dict
    
    ### execute whole process
    def execute_scrapping(self):
        re_list = []
        stopper = False
        counter = 1

        while stopper != True:
            page_n = self.get_search_pages(self.search_term , counter)
            page_source = page_n.find_all('div' , attrs = {'class' : 'shop-sku-list-item'})
            
            if len(page_source) == 0:
                stopper = True
            else:
                for product in page_source:
                    re_list.append(self.get_product_info(product))
                print(f"page {counter} done!")
                counter = counter + 1
            
            ### pause after request
            time.sleep(1)
        ### return dict of products
        return re_list

In [12]:
User_Agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36 Edg/88.0.705.56'
p30 = requests.get("https://www.bestbuy.com/site/searchpage.jsp?st=TV&cp=29" , headers = {'User-Agent' : User_Agent})
bs_p30 = BeautifulSoup(p30.content , "lxml")

In [29]:
bs_p30

<!DOCTYPE HTML>
<html lang="en">
<head>
<!-- platview-njs: version=23.10.1 | blueprint: id=plp-search-l-a | version: id=11 -->
<!-- start simple component shop-plp-seo-metadata-5765465 - shop-plp-seo-metadata v2 -->
<title>tv - Best Buy</title>
<meta content="IE=Edge,chrome=1" http-equiv="X-UA-Compatible"/><meta content="telephone=no" name="format-detection"/><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="no-cache" http-equiv="pragma"/><meta content="no-cache" http-equiv="cache-control"/><meta content="-1" http-equiv="expires"/><meta content="width=1024" name="viewport"/><meta content="Shop for tv at Best Buy. Find low everyday prices and buy online for delivery or in-store pick-up" name="description"/><meta content="tv" name="keywords"/><meta content='{"query":"TV","queryType":"search"}' name="opt-targeting"/>
<link href="//www.bestbuy.com/favicon.ico" rel="icon" type="image/x-icon"/><link href="https://www.bestbuy.com/site/searchpage.jsp?id=pcat170

In [62]:
bs_p30.find_all("li" , attrs = {'class' : 'sku-item'})[7].text

'Package - Insignia™ - 32" Class N10 Series LED HD TV - Black and Google - Chromecast with Google TV (HD) - SnowUser rating, 4.6 out of 5 stars with 3825 reviews.(3,825)Save$124.98Your price for this item is $124.98Package PriceReg $159.98The regular price is $159.98'

In [18]:
if __name__ == '__main__': 
    
    ### initiate object
    bb_init = Best_buy_scrap("TV")
    
    ### start scrapping
    dict_info = bb_init.execute_scrapping()

https://www.bestbuy.com/site/searchpage.jsp?st=TV&cp=1
page 1 done!
https://www.bestbuy.com/site/searchpage.jsp?st=TV&cp=2
page 2 done!
https://www.bestbuy.com/site/searchpage.jsp?st=TV&cp=3
page 3 done!
https://www.bestbuy.com/site/searchpage.jsp?st=TV&cp=4
page 4 done!
https://www.bestbuy.com/site/searchpage.jsp?st=TV&cp=5
page 5 done!
https://www.bestbuy.com/site/searchpage.jsp?st=TV&cp=6
page 6 done!
https://www.bestbuy.com/site/searchpage.jsp?st=TV&cp=7
page 7 done!
https://www.bestbuy.com/site/searchpage.jsp?st=TV&cp=8
page 8 done!
https://www.bestbuy.com/site/searchpage.jsp?st=TV&cp=9
page 9 done!
https://www.bestbuy.com/site/searchpage.jsp?st=TV&cp=10
page 10 done!
https://www.bestbuy.com/site/searchpage.jsp?st=TV&cp=11
page 11 done!
https://www.bestbuy.com/site/searchpage.jsp?st=TV&cp=12
page 12 done!
https://www.bestbuy.com/site/searchpage.jsp?st=TV&cp=13
page 13 done!
https://www.bestbuy.com/site/searchpage.jsp?st=TV&cp=14
page 14 done!
https://www.bestbuy.com/site/searchpag

In [19]:
### check results
dict_info

[{'header': 'Samsung - 65" Class 7 Series LED 4K UHD Smart Tizen TV',
  'brand': 'Samsung',
  'labels': ['4K UHD', 'Direct Lit', 'Smart'],
  'model': 'UN65TU7000FXZA',
  'sku': '6401722',
  'stars': 4.6,
  'number_of_reivews': '12679',
  'price': 479.99},
 {'header': 'Insignia™ - 55" Class F30 Series LED 4K UHD Smart Fire TV',
  'brand': 'Insignia™',
  'labels': ['4K UHD', 'Direct Lit', 'Smart', 'Voice Assist'],
  'model': 'NS-55F301NA22',
  'sku': '6450247',
  'stars': 4.6,
  'number_of_reivews': '3078',
  'price': 269.99},
 {'header': 'Samsung - 55" Class 7 Series LED 4K UHD Smart Tizen TV',
  'brand': 'Samsung',
  'labels': ['4K UHD', 'Direct Lit', 'Smart'],
  'model': 'UN55TU7000FXZA',
  'sku': '6401735',
  'stars': 4.6,
  'number_of_reivews': '8788',
  'price': 379.99},
 {'header': 'LG - 70" Class NanoCell 75UQA Series LED 4K UHD Smart webOS TV',
  'brand': 'LG',
  'labels': ['4K UHD', 'Direct Lit', 'Smart', 'Voice Assist'],
  'model': '70NANO75UQA',
  'sku': '6501916',
  'stars':

In [68]:
# Connect to MongoDB database
client = MongoClient("personal_info")

# Get the "bayc" collection
db = client["best_buy_db"]

# Drop the "bayc" collection if it exists
if "best_buy_collection" in db.list_collection_names():
    db["best_buy_collection"].drop()

# create a db 
collection = db["best_buy_collection"]



In [69]:
#pushing into collection
collection.insert_many(dict_info)